<a href="https://colab.research.google.com/github/davidivan13/Python-for-Data-Science/blob/main/TensorFlow_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Datasets

TFDS menyediakan kumpulan dataset siap pakai untuk digunakan dengan TensorFlow, Jax, dan kerangka pembelajaran mesin lainnya.

Ini menangani pengunduhan dan persiapan data secara deterministik dan membangun `tf.data.Dataset` (atau `np.array`).

Catatan: Jangan bingung antara [TFDS](https://www.tensorflow.org/datasets) (perpustakaan ini) dengan `tf.data` (API TensorFlow untuk membangun pipa data efisien). TFDS adalah lapisan tinggi yang melibatkan `tf.data`. Jika Anda tidak familiar dengan API ini, kami sarankan membaca [panduan resmi tf.data](https://www.tensorflow.org/guide/data) terlebih dahulu.


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Dataset yang tersedia di Tensorflow

Semua builder dataset adalah turunan dari `tfds.core.DatasetBuilder`. Untuk mendapatkan daftar pembangun yang tersedia, gunakan `tfds.list_builders()` atau lihat [katalog dataset](https://www.tensorflow.org/datasets/catalog/overview).

In [2]:
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'ai2dcaption',
 'aloha_mobile',
 'amazon_us_reviews',
 'anli',
 'answer_equivalence',
 'arc',
 'asimov_dilemmas_auto_val',
 'asimov_dilemmas_scifi_train',
 'asimov_dilemmas_scifi_val',
 'asimov_injury_val',
 'asimov_multimodal_auto_val',
 'asimov_multimodal_manual_val',
 'asqa',
 'asset',
 'assin2',
 'asu_table_top_converted_externally_to_rlds',
 'austin_buds_dataset_converted_externally_to_rlds',
 'austin_sailor_dataset_converted_externally_to_rlds',
 'austin_sirius_dataset_converted_externally_to_rlds',
 'bair_robot_pushing_small',
 'bc_z',
 'bccd',
 'beans',
 'bee_dataset',
 'beir',
 'berkeley_autolab_ur5',
 'berkeley_cable_routing',
 'berkeley_fanuc_manipulation',
 'berkeley_gnm_cory_hall',
 'berkeley_gnm_recon',
 'berkeley_gnm_sac_son',
 'berkeley_mvp_converted_externally_to_rlds',
 'berkeley_rpt_converted_externally_to_rlds',
 'big_patent',
 'bigearthnet',
 'billsum',
 '

## Bangun end to end pipeline

Demonstrasi cara pembuatan tensorflow dataset untuk training model.


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds

### Step 1: Bangun input pipeline

referensi membangun input pipeline yang efisien:
* The [Performance tips](https://www.tensorflow.org/datasets/performances) guide
* The [Better performance with the `tf.data` API](https://www.tensorflow.org/guide/data_performance#optimize_performance) guide

#### Load a dataset

Kita akan menggunakan data MNIST:

* `shuffle_files=True`:Data MNIST hanya disimpan dalam satu file, tetapi untuk dataset yang lebih besar dengan beberapa file di disk, praktik yang baik adalah untuk mengacaknya saat pelatihan.
* `as_supervised=True`: Returns tuple `(img, label)` bukan dictionary `{'image': img, 'label': label}`.

In [4]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.I20KBN_3.0.1/mnist-train.tfrecord*...:   0%|          | 0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.I20KBN_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.




Terapkan transformasi berikut:

* `tf.data.Dataset.map`: TFDS menyediakan gambar dengan tipe `tf.uint8`, sementara model mengharapkan tipe `tf.float32`. Oleh karena itu, Anda perlu normalisasi gambar.
* `tf.data.Dataset.cache`: Saat Anda memuat dataset ke dalam memori, simpan di cache sebelum diacak untuk kinerja yang lebih baik.<br/>
__Catatan:__ Transformasi acak harus diterapkan setelah penyimpanan di cache.
* `tf.data.Dataset.shuffle`: Untuk keacakan sejati, atur buffer pengacakan sebesar ukuran dataset penuh.<br/>
__Catatan:__ Untuk dataset besar yang tidak muat dalam memori, gunakan `buffer_size=1000` jika sistem Anda mendukungnya.
* `tf.data.Dataset.batch`: Gabungkan elemen dataset setelah diacak untuk mendapatkan batch unik pada setiap epoch.
* `tf.data.Dataset.prefetch`: Praktik yang baik untuk mengakhiri pipa data dengan prefetching [untuk kinerja](https://www.tensorflow.org/guide/data_performance#prefetching).

## Training Set

In [5]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

## Evaluation Set

Pipeline data pengujian Anda mirip dengan pipeline data pelatihan dengan perbedaan kecil:

* Anda tidak perlu memanggil `tf.data.Dataset.shuffle`.
* Penyimpanan di cache dilakukan setelah pengelompokan karena batch dapat sama antara epoch.

In [6]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

### Step 2: Build and Train Model
Gunakan train set dan test set untuk melatih modelnya

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation='relu'),  # Increased the number of neurons for better representation
    tf.keras.layers.Dropout(0.5),  # Added dropout for regularization
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Added dropout for regularization
    tf.keras.layers.Dense(10)
])

# Compile the optimized model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Explicitly set learning rate
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - loss: 0.8696 - sparse_categorical_accuracy: 0.7146 - val_loss: 0.1742 - val_sparse_categorical_accuracy: 0.9458
Epoch 2/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2562 - sparse_categorical_accuracy: 0.9249 - val_loss: 0.1251 - val_sparse_categorical_accuracy: 0.9623
Epoch 3/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.1967 - sparse_categorical_accuracy: 0.9430 - val_loss: 0.1038 - val_sparse_categorical_accuracy: 0.9681
Epoch 4/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.1614 - sparse_categorical_accuracy: 0.9528 - val_loss: 0.0942 - val_sparse_categorical_accuracy: 0.9729
Epoch 5/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1500 - sparse_categorical_accuracy: 0.9562 - val_loss: 0.0865 - val_sparse_categorical_accuracy: 0.9745
Epoch 6/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1270 - sparse_categorical_accuracy: 0.9631 - val_loss: 0.0846 - val_sparse_categorical_accuracy: 0.9750
